In [34]:
import pandas as pd
df = pd.read_csv('../data/2021玉山人工智慧公開挑戰賽冬季賽訓練資料集.gz.gz', compression='gzip')
df.columns = ['dt'] + df.columns[1:].tolist() # 更改欄位名稱
test_data = pd.read_csv('..\data\需預測的顧客名單及提交檔案範例.csv')
print(train_data.shape,test_data.shape)
display(train_data.head())
display(test_data.head())

(32475654, 53) (500000, 4)


,dt,chid,shop_tag,txn_cnt,txn_amt,domestic_offline_cnt,domestic_online_cnt,overseas_offline_cnt,overseas_online_cnt,domestic_offline_amt_pct,...,masts,educd,trdtp,naty,poscd,cuorg,slam,gender_code,age,primary_card
0,1.0,10321418.0,45,3.0,3891.965283,3.0,0.0,0.0,0.0,1.00,...,1.0,4.0,5.0,1.0,99.0,30.0,95982.822967,1.0,4.0,1.0
1,1.0,10414574.0,15,2.0,10616.561549,2.0,0.0,0.0,0.0,1.00,...,2.0,2.0,15.0,1.0,2.0,30.0,130702.351368,1.0,3.0,1.0
2,1.0,10134567.0,48,2.0,23527.655416,0.0,2.0,0.0,0.0,0.00,...,1.0,3.0,11.0,1.0,3.0,30.0,112010.611717,0.0,4.0,1.0
3,1.0,10001003.0,48,9.0,17751.558260,7.0,2.0,0.0,0.0,0.75,...,2.0,3.0,11.0,1.0,2.0,30.0,59701.507360,0.0,3.0,1.0
4,1.0,10267183.0,2,1.0,21701.307598,1.0,0.0,0.0,0.0,1.00,...,2.0,6.0,15.0,1.0,99.0,30.0,NaN,0.0,6.0,0.0


,chid,top1,top2,top3
0,10128239,18,10,6
1,10077943,48,22,6
2,10277876,22,10,39
3,10364842,37,19,13
4,10392717,2,36,26


In [78]:
feature_columns = list(set(train_data.columns.tolist()) - set(['chid','shop_tag']))
categorical_features = ['masts','educd','trdtp','naty','poscd','cuorg','gender_code','age','primary_card']
numerical_features = list(set(feature_columns)-set(categorical_features))
train_df = train_data
sample_submit = test_data
len(feature_columns),len(categorical_features),len(numerical_features)

(51, 9, 42)

In [79]:
from tqdm.notebook import tqdm
for idx in tqdm(sample_submit.index[:3]): # demo用所以取前五筆 實際上應該取全部
    # 取出這一條row的chid
    chid = sample_submit.loc[idx,'chid']
    
    # 從歷史訓練數據抓對應的chid 算一些統計資訊 當作特徵 併入sample_submit裡面

    # 數值特徵統計資訊
    for col in tqdm(numerical_features):
        # 1.平均值  2.最大值  3.最小值 
        sample_submit.loc[idx,f"mean_of_{col}"] = train_df.loc[train_df['chid']==chid,col].mean()
        sample_submit.loc[idx,f"max_of_{col}"] = train_df.loc[train_df['chid']==chid,col].max()
        sample_submit.loc[idx,f"min_of_{col}"] = train_df.loc[train_df['chid']==chid,col].min()
        
    # 類別特徵統計資訊
    for col in tqdm(categorical_features):
        # 1.nunique (這個欄位有多少種不一樣的數值)
        sample_submit.loc[idx,f"nunique_of_{col}"] = train_df.loc[train_df['chid']==chid,col].nunique()
        # 2.norm_count (這個欄位在每一種數值上出現的次數(count)除以總數(normalize)等於取平均(mean))
        cat_feat = pd.get_dummies(train_df[col],columns=col) # one hot 格式
        cat_feat = pd.DataFrame(cat_feat.mean(axis=0)).T # 取平均
        cat_feat.index = [idx] # 處理DataFrame格式
        cat_feat.columns = [ "norm_count_"+col+"_"+str(i) for i in cat_feat.columns] # 處理DataFrame格式
        sample_submit.loc[[idx]] = sample_submit.loc[[idx]].join(cat_feat) # 合併DataFrame

sample_submit.head(3)

,chid,top1,top2,top3,mean_of_masts,nunique_of_masts,mean_of_educd,nunique_of_educd,mean_of_trdtp,nunique_of_trdtp,...,min_of_domestic_offline_amt_pct,mean_of_card_11_txn_cnt,max_of_card_11_txn_cnt,min_of_card_11_txn_cnt,mean_of_card_8_txn_amt_pct,max_of_card_8_txn_amt_pct,min_of_card_8_txn_amt_pct,nunique_of_gender_code,nunique_of_age,nunique_of_primary_card
0,10128239,18,10,6,1.0,1.0,4.0,1.0,8.0,1.0,...,1.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0
1,10077943,48,22,6,1.0,1.0,5.0,1.0,12.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0
2,10277876,22,10,39,1.0,1.0,5.0,1.0,6.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [81]:
sample_submit.head(10)

,chid,top1,top2,top3,mean_of_masts,nunique_of_masts,mean_of_educd,nunique_of_educd,mean_of_trdtp,nunique_of_trdtp,...,min_of_domestic_offline_amt_pct,mean_of_card_11_txn_cnt,max_of_card_11_txn_cnt,min_of_card_11_txn_cnt,mean_of_card_8_txn_amt_pct,max_of_card_8_txn_amt_pct,min_of_card_8_txn_amt_pct,nunique_of_gender_code,nunique_of_age,nunique_of_primary_card
0,10128239,18,10,6,1.0,1.0,4.0,1.0,8.0,1.0,...,1.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0
1,10077943,48,22,6,1.0,1.0,5.0,1.0,12.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0
2,10277876,22,10,39,1.0,1.0,5.0,1.0,6.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,10364842,37,19,13,2.0,1.0,5.0,1.0,7.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,10392717,2,36,26,2.0,1.0,3.0,1.0,5.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,10439550,22,26,18,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,10285755,12,13,36,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,10335334,26,18,48,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,10072243,15,13,6,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,10274006,22,6,39,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [82]:
print(sample_submit.shape)
sample_submit.describe()

(500000, 145)


,chid,top1,top2,top3,mean_of_masts,nunique_of_masts,mean_of_educd,nunique_of_educd,mean_of_trdtp,nunique_of_trdtp,...,min_of_domestic_offline_amt_pct,mean_of_card_11_txn_cnt,max_of_card_11_txn_cnt,min_of_card_11_txn_cnt,mean_of_card_8_txn_amt_pct,max_of_card_8_txn_amt_pct,min_of_card_8_txn_amt_pct,nunique_of_gender_code,nunique_of_age,nunique_of_primary_card
count,5.000000e+05,500000.000000,500000.000000,500000.000000,5.000000,5.0,5.000000,5.0,5.000000,5.0,...,3.000000,3.0,3.0,3.0,3.000000,3.000000,3.000000,3.000000,3.000000,3.0
mean,1.025000e+07,21.805368,21.821710,21.815982,1.400000,1.0,4.400000,1.0,7.600000,1.0,...,0.333333,0.0,0.0,0.0,0.333333,0.333333,0.333333,0.666667,0.666667,1.0
std,1.443377e+05,12.417363,12.418025,12.407671,0.547723,0.0,0.894427,0.0,2.701851,0.0,...,0.577350,0.0,0.0,0.0,0.577350,0.577350,0.577350,0.577350,0.577350,0.0
min,1.000000e+07,2.000000,2.000000,2.000000,1.000000,1.0,3.000000,1.0,5.000000,1.0,...,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,1.0
25%,1.012500e+07,12.000000,13.000000,13.000000,1.000000,1.0,4.000000,1.0,6.000000,1.0,...,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.500000,0.500000,1.0
50%,1.025000e+07,19.000000,19.000000,21.000000,1.000000,1.0,5.000000,1.0,7.000000,1.0,...,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,1.000000,1.000000,1.0
75%,1.037500e+07,36.000000,36.000000,36.000000,2.000000,1.0,5.000000,1.0,8.000000,1.0,...,0.500000,0.0,0.0,0.0,0.500000,0.500000,0.500000,1.000000,1.000000,1.0
max,1.050000e+07,48.000000,48.000000,48.000000,2.000000,1.0,5.000000,1.0,12.000000,1.0,...,1.000000,0.0,0.0,0.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.0


In [85]:
pd.options.display.max_rows = 999
a = pd.DataFrame()
a['features_name'] = sample_submit.columns.tolist()
a

,features_name
0,chid
1,top1
2,top2
3,top3
4,mean_of_masts
5,nunique_of_masts
6,mean_of_educd
7,nunique_of_educd
8,mean_of_trdtp
9,nunique_of_trdtp


# 保存

In [86]:
#train_data.reset_index().to_feather('../data/train_data.feather')
#test_data.reset_index().to_feather('../data/test_data.feather')